# Softening Pre-Treatment: Lime and Soda Ash

References:

1. Lee, C. C., & Lin, S. D. (2007). Handbook of environmental engineering
   calculations. New York: McGraw Hill.
   
2. Crittenden, J. C., & Montgomery Watson Harza (Firm). (2012). Water treatment principles and design. Hoboken, N.J: J.Wiley.

3. Davis, M. L. (2010). Water and wastewater engineering: Design principles and practice.

4. Baruth. (2005). Water treatment plant design / American Water Works Association, American Society of Civil Engineers; 
   Edward E. Baruth, technical editor. (Fourth edition.). McGraw-Hill.
   
5. Edzwald, J. K., & American Water Works Association. (2011). Water quality & treatment: A handbook on drinking water. 
   New York: McGraw-Hill.

6. R.O. Mines Environmental Engineering: Principles and Practice, 1st Ed, John Wiley & Sons

7. McGivney, W. T. & Kawamura, S. (2008) Cost Estimating Manual for Water Treatment Facilities. John Wiley & Sons, Inc., Hoboken, NJ, USA.

Pending Questions:

    - What to do about the density calculation based on Temperature?
    
    - The calculation of excess lime is possible, yet, the program will not be a 0 DOF model. Mostly due to the variable x1. 
      The fixing of pH_modif makes solve unfeasible. A calculation of 15% excess lime was adapted in the mean time.
      
    - Do chemical costs (i.e., MgO, Soda Ash, etc.) need to be obtained from a specific source?
    
    - Is there a way that the program will automatically update costs from 2009 to the current date 2022 and beyond?
    
    - CO2 estimation issues with K1, K2** and CT calculations -- Fixed with specifying bounds as (None, None)
    
    - Bounds for parameters velocity gradients in Rapid Mix and Flocculation?
    
    - Costs expressions in CAPEX and OPEX, or in $/m3 and $/kgal?
    
    - Unit issues - What if the user wants to use US units? Example, instead of user providing flow in m3/d, provides MGD.
    

In [2]:
from pyomo.environ import *
from pyomo.environ import units as pyunits
from pyomo.util.check_units import assert_units_consistent
from idaes.core.util.model_statistics import degrees_of_freedom
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# from iapws import IAPWS97
import pyomo.util.infeasible as infeas
# from watertap.core.util.infeasible import *
# from idaes.core.util import get_solver
from idaes.core.solvers.get_solver import get_solver
import idaes.core.util.scaling as iscale
# print_infeasible_constraints(m)
# print_infeasible_bounds(m)
# print_variables_close_to_bounds(m)
# print_constraints_close_to_bounds(m)
# print_close_to_bounds(m)
# print(infeas.log_infeasible_bounds(m))
# print(infeas.log_infeasible_constraints(m))

In [6]:
#hard-coded values for testing
Q=50000 # very high flows prove unfeasible above 800,000 m3/d == 212 MGD
pH=7.0 # pH above 8.2 and below 7.2 -- not feasible when no CO2 is provided -- Var bounds changed None, none - No problems
Ca=75
Mg=6.1
Alk=196
Si=0
CO2=72 # CO2 estimation calculations are having some trouble -- discuss with Kurby -- Somewhat fixed with modifying the bounds
TSS=200
T=10 # T has an influence in model solve when no CO2 is provided
#ExCaOH=30
rt_r=0.4
rt_f=25
rt_s=130
rt_rec=20
g_r=300
g_f=50
num_r=1
num_f=2

#if silica removal True  --- Boolean approach for user
silica_removal = True

##### Model Build 

In [7]:
m = ConcreteModel(name="-- Softening Pre-Treatment")

# Parameters ---------------------------------------------------------------------------

m.Q = Param(initialize=Q,
            units=pyunits.m**3/pyunits.day,
            doc='Water flow in m^3/day')

m.T = Param(initialize=T,
            units=pyunits.C,
            doc='Temperature of water in C')

m.T_K = Param(initialize=T+273.15,
              units=pyunits.K,
              doc='Temperature of water in K')

m.Ca = Param(initialize=Ca,
             doc='Calcium influent in mg/L')

m.Mg = Param(initialize=Mg,
             doc='Magnesium influent in mg/L')

m.Alk = Param(initialize=Alk,
              doc='Bicarbonate influent in mg/L as CaCO3')

m.Si = Param(initialize=Si,
             doc='Silica influent in mg/L')

m.CO2 = Param(initialize=CO2,
             doc='CO2 influent in mg/L')

m.TSS = Param(initialize=TSS,
              units=pyunits.mg / pyunits.L,
              doc='TSS influent in mg/L')

m.mu = Param(initialize=0.001) # CHECK IAPWS????30599

m.Ca_CaCO3_conv = Param(initialize=2.5,
                        units=pyunits.dimensionless,
                        doc='Conversion factor for Ca mg/L to mg/L as CaCO3')

m.Mg_CaCO3_conv = Param(initialize=4.12,
                        units=pyunits.dimensionless,
                       doc='Conversion factor for Mg mg/L to mg/L as CaCO3')

m.Ca_CaCO3 = Param(initialize=Ca*m.Ca_CaCO3_conv,
                   doc='Calcium influent in mg/L as CaCO3')

m.Mg_CaCO3 = Param(initialize=Mg*m.Mg_CaCO3_conv,
                   doc='Magnesium influent in mg/L as CaCO3')

m.TH = Param(initialize=m.Ca_CaCO3+m.Mg_CaCO3,
             doc='Total Hardness influent in mg/L as CaCO3')

m.G_R = Param(initialize=g_r,
            doc='Velocity gradient of rapid mix tank') # BOUNDS NEEDED

m.G_F = Param(initialize=g_f,
            doc='Velocity gradient of flocculation tank') # BOUNDS NEEDED

# Variables-----------------------------------------------------------

m.pH = Var(initialize=pH,
           within=NonNegativeReals,
           bounds=(0,14))

m.Ca_eff = Var(initialize=30,
               units=pyunits.mg / pyunits.L,
               within=NonNegativeReals,
               bounds=(10,60),
               doc='Calcium effluent in mg/L as CaCO3')

m.Mg_eff = Var(initialize=10,
               units=pyunits.mg / pyunits.L,
               within=NonNegativeReals,
               bounds=(5,40),
               doc='Magnesium effluent in mg/L as CaCO3')

m.K1 = Var(initialize=1E-7,
           bounds=(None, None), # changed to none-none
           doc='K1 value for CO2 estimation')

m.K2 = Var(initialize=1E-10,
           bounds=(None, None), # changed to none-none
           doc='K2 value for CO2 estimation')

m.Alpha_1 = Var(initialize=0.75,
               bounds=(None, None), # changed to none-none
               doc='Alpha value for CO2 estimation')

m.CT_1 = Var(initialize=0.0025,
            bounds=(None, None), # changed to none-none
            doc='CT1 value for CO2 estimation')

m.CO2_CaCO3 = Var(initialize=115,
                  units=pyunits.mg / pyunits.L,
                  bounds=(None, None),
                  doc='Carbonic acid concentration in mg/L as CaCO3')

m.pKso = Var(initialize=10.15,
             bounds=(0, None),
             doc='Solubility product constant for MgOH2')

m.pKw = Var(initialize=1E-14,
            bounds=(0, None),
            doc='Equilibrium - Kw')
"""
m.Mg_eff_mol = Var(initialize=1E-4,
                   bounds=(0, None),
                   doc='Mg effluent concentration specified (or default = 10 mg/L)')

m.OH_c1 = Var(initialize=0.00084025,
            bounds=(0, None),
              doc='OH- concentration testing one')

m.pH_MgRem = Var(initialize=10,
                bounds=(6, 12),
                 doc='pH in our process to obtain a specified (or default = 10 mg/L) Mg effluent value')
"""
m.x1 = Var(initialize=2.53,
           bounds=(None, None),
           doc='Excess lime - OH counter varable')

m.pH_modif = Var(initialize=11,
                bounds=(11, 12),
                 doc='pH modification for excess lime dosage requirement')

m.exCaOH = Var(initialize=30,
               units=pyunits.mg / pyunits.L,
               bounds=(0, None),
               doc='Estimated excess lime requirement in mg/L of CaOH -- accounting for silica removal pH of 11.5')

"""
m.CH = Var(initialize=195,
           bounds=(0, None),
           doc='Carbonate hardness in mg/L as CaCO3')

m.NCH = Var(initialize=17.5,
            bounds=(0, None),
            doc='Non-carbonate hardness in mg/L as CaCO3')
"""

m.CHCa = Var(initialize=187.5,
             units=pyunits.mg / pyunits.L,
             bounds=(0, None),
             doc='Calcium carbonate hardness in mg/L as CaCO3')

m.NCHCa = Var(initialize=17.5,
              units=pyunits.mg / pyunits.L,
              bounds=(0, None),
              doc='Calcium non-carbonate hardness in mg/L as CaCO3')

m.CHMg = Var(initialize=7.5,
             units=pyunits.mg / pyunits.L,
             bounds=(0, None),
             doc='Magnesium carbonate hardness in mg/L as CaCO3')

m.NCHMg = Var(initialize=17.5,
              units=pyunits.mg / pyunits.L,
              bounds=(None, None),
              doc='Magnesium non-carbonate hardness in mg/L as CaCO3')

m.CaOH = Var(initialize=1e5,
             units=pyunits.kg / pyunits.day,
             bounds=(0, None),
             doc='Lime concentration required of CaOH/CaO ')

m.Na2CO3 = Var(initialize=1e5,
               units=pyunits.kg / pyunits.day,
               bounds=(0, None),
               doc='Soda ash concentration required of Na2CO3')

m.MgCl2 = Var(initialize=190,
              units=pyunits.kg / pyunits.day,
              bounds=(0, None),
              doc='Magnesium chloride concentration required of MgCl2')

m.CO2_req1 = Var(initialize=1E2,
                 units=pyunits.kg / pyunits.day,
                 bounds=(0, None),
                 doc='CO2 concentration required of CO2 (a)')

m.CO2_req2 = Var(initialize=1E2,
                 units=pyunits.kg / pyunits.day,
                 bounds=(0, None),
                 doc='CO2 concentration required of CO2 (b)')

m.Sludge = Var(initialize=1000000,
               units=pyunits.kg / pyunits.day,
               bounds=(0, None),
               doc='Sludge production in kg/day')

m.RT_R = Var(initialize=rt_r,
             bounds=(0.1, 5),
             units=pyunits.minutes,
             doc='Retention time of rapid mix tank')

m.RT_F = Var(initialize=rt_f,
             bounds=(10, 45),
             units=pyunits.minutes,
             doc='Retention time of flocculation tank')

m.RT_S = Var(initialize=rt_s,
             bounds=(120, 240),
             units=pyunits.minutes,
             doc='Retention time of sedimentation basin')

m.D_S = Var(initialize=4.5,
             bounds=(3, 6),
             units=pyunits.m,
             doc='Depth of sedimentation basin')

m.RT_Rec = Var(initialize=rt_rec,
               bounds=(15, 30),
               units=pyunits.minutes,
               doc='Retention time of recarbonation basin')

m.Num_R = Var(initialize=num_r,
              bounds=(1, 3),
              units=pyunits.dimensionless,
              doc='Number of rapid mix tanks')

m.Num_F = Var(initialize=num_f,
              bounds=(2, 6),
              units=pyunits.dimensionless,
              doc='Number of flocculation tanks')

m.V1 = Var(initialize=15,
           bounds=(0, None),
           units=pyunits.m**3,
           doc='Volume of rapid mix tank')

m.V2 = Var(initialize=75,
           bounds=(0, None),
           units=pyunits.m**3,
           doc='Volume of flocculation tank')

m.V3 = Var(initialize=100,
           bounds=(0, None),
           units=pyunits.m**3,
           doc='Volume of sedimentation basin')

m.V4 = Var(initialize=45,
           bounds=(0, None),
           units=pyunits.m**3,
           doc='Volume of recarbonation basin')

m.E1 = Var(initialize=1000,
           bounds=(0, None),
           units=pyunits.W,
           doc='Rapid mix energy requirement')

m.E2 = Var(initialize=100,
           bounds=(0, None),
           units=pyunits.W,
           doc='Flocculation energy requirement')

m.rapid_mix_power = Var(initialize=100,
                       units=pyunits.kWh/pyunits.day,
                       doc='Power consumption for rapid mix')

m.floc_power = Var(initialize=100,
                       units=pyunits.kWh/pyunits.day,
                       doc='Power consumption for rapid mix')

m.C1 = Var(initialize=1E6,
           bounds=(0, None),
           units=pyunits.dimensionless,
           doc='Capital Cost of rapid mix tank')

m.C2 = Var(initialize=1E6,
           bounds=(0, None),
           units=pyunits.dimensionless,
           doc='Capital Cost of flocculation tank')

m.C3 = Var(initialize=1E6,
           bounds=(0, None),
           units=pyunits.dimensionless,
           doc='Capital Cost of sedimentation basin - Circular basin')

m.C4 = Var(initialize=1E6,
           bounds=(0, None),
           units=pyunits.dimensionless,
           doc='Capital Cost of recarbonation basin')

m.C5 = Var(initialize=1E6,
           bounds=(0, None),
           units=pyunits.dimensionless,
           doc='Capital Cost of recarbonation - Liquid CO2 as CO2 source')

m.C6 = Var(initialize=1E6,
           bounds=(0, None),
           units=pyunits.dimensionless,
           doc='Capital Cost of lime feed system')

m.C7 = Var(initialize=1E6,
           bounds=(0, None),
           units=pyunits.dimensionless,
           doc='Capital Cost of administrative, laboratory, and maintenance building')

m.OC1 = Var(initialize=1E4,
            bounds=(0, None),
            units=pyunits.dimensionless,
            doc='Operational Cost of rapid mix tank -- energy consumption and labor')

m.OC2 = Var(initialize=1E4,
            bounds=(0, None),
            units=pyunits.dimensionless,
            doc='Operational Cost of flocculation tank -- energy consumption and labor')

m.OC3 = Var(initialize=1E4,
            bounds=(0, None),
            doc='Operational Cost of sedimentation basin -- energy consumption, labor, and maintenance')

m.OC4 = Var(initialize=1E4,
            bounds=(0, None),
            units=pyunits.dimensionless,
            doc='Operational Cost of recarbonation basin -- energy consumption, labor, and maintenance')

m.OC5 = Var(initialize=1E4,
            bounds=(0, None),
            units=pyunits.dimensionless,
            doc='Operational Cost of lime feed system -- energy consumption, labor, and maintenance')


m.OC6 = Var(initialize=1E4,
            bounds=(0, None),
            units=pyunits.dimensionless,
            doc='Operational Cost of soda ash -- chemical addition cost')

m.OC7 = Var(initialize=1E4,
            bounds=(0, None),
            units=pyunits.dimensionless,
            doc='Operational Cost of magnesium -- chemical addition cost')

m.OC8 = Var(initialize=1E4,
            bounds=(0, None),
            units=pyunits.dimensionless,
            doc='Operational Cost lime sludge management -- apporximate based on per ton of lime')


m.OC9 = Var(initialize=1E4,
            bounds=(0, None),
            units=pyunits.dimensionless,
            doc='Operational Cost of administrative, laboratory, and maintenance building')

m.CAPEX = Var(initialize=3E5,
              bounds=(0, None),
              units=pyunits.dimensionless,
              doc='Total Capital Expenses in Treatment System')

m.OPEX = Var(initialize=5E4,
             bounds=(0, None),
              units=pyunits.dimensionless,
              doc='Total Operational Expenses in Treatment System')


# Constraint equations
#---------------------------------------------------------------------------------------------------------------

if m.CO2 == 0:
    m.K1_constr = Constraint(expr=m.K1 == 10 ** (14.8435 - (3404.71 / m.T_K) - (0.032786 * m.T_K)))
    m.K2_constr = Constraint(expr=m.K2 == 10 ** (6.498 - (2909.39 / m.T_K) - (0.02379 * m.T_K)))
    m.Alpha_1_constr = Constraint(expr=m.Alpha_1 == 1 /(((10 ** -m.pH) / m.K1) + 1 + (m.K2 / (10 ** - m.pH))))
    m.CT_1_constr = Constraint(expr=m.CT_1 == (m.Alk * 0.01 * 0.001) / m.Alpha_1)
    m.CO2_CaCO3_constr = Constraint(expr=m.CO2_CaCO3 == (((m.CT_1 - (m.Alk*0.00001))*100000) * 1.612))
else:
    m.CO2_CaCO3_constr = Constraint(expr=m.CO2_CaCO3 == m.CO2 * 1.612)
    
#if statement for hardness determination

"""
NOT NEEDED-----------------------
if m.TH <= m.Alk:
    print('a')
    m.CH_constr = Constraint(expr=m.CH == m.TH)
    m.NCH.fix(0)
    #m.CH.fix()
else:
    print('b')
    m.CH_constr = Constraint(expr=m.CH == m.Alk)
    m.NCH_constr = Constraint(expr=m.NCH == m.TH - m.Alk)
    #m.CH.fix()
    #m.NCH.fix()
----------------------------------
"""

if m.Ca_CaCO3 >= m.Alk:
    #print('aa')
    m.CHCa_constr = Constraint(expr=m.CHCa == m.Alk)
    m.NCHCa_constr = Constraint(expr=m.NCHCa == m.Ca_CaCO3 - m.Alk)
    m.CHMg.fix(0)
    m.NCHMg_constr = Constraint(expr=m.NCHMg == m.Mg_CaCO3)
else:
    #print('bb')
    m.CHCa_constr = Constraint(expr=m.CHCa == m.Ca_CaCO3)
    m.NCHCa.fix(0)
    m.CHMg_constr = Constraint(expr=m.CHMg == m.Alk - m.Ca_CaCO3)
    m.NCHMg_constr = Constraint(expr=m.NCHMg == m.TH - m.Alk)

"""
# CHECK--------------------------------------------------------------------------------------
# Not working---- Can be simplified
m.pKso_constr = Constraint(expr=m.pKso == 0.0175 * (m.T) + 9.97)
m.pKw_constr = Constraint(expr=m.pKw == (4470.99 / m.T_K) + 0.017060 * m.T_K - 6.0875)
m.Mg_eff_mol_constr = Constraint(expr=m.Mg_eff_mol == (m.Mg_eff / 100) * 0.001)
m.OH_c1_constr = Constraint(expr=m.OH_c1 == sqrt((10 ** -m.pKso) / m.Mg_eff_mol))
m.pH_MgRem_constr = Constraint(expr=m.pH_MgRem == 14 + log10(m.OH_c1))
#print(m.pH_MgRem())
m.pH_modif_constr = Constraint(expr=m.pH_modif == -log10(10 ** -m.pKw / 10 ** -m.x1) >= 11.5)
#print(m.pH_modif())
m.exCaOH_constr = Constraint(expr=m.exCaOH == 10 ** (-m.x1) / 2 * 74000)
#print(m.exCaOH())


NEW --- Appears to work but leaves 1 DOF --- When fixing the m.pH_modif ==> unfeasible solve
if silica_removal:
    m.pKw_constr = Constraint(expr=m.pKw == (4470.99 / m.T_K) + 0.017060 * m.T_K - 6.0875)
    m.pH_new_constr = Constraint(expr=m.pH_modif >= 11.5)
    m.pH_modif_constr = Constraint(expr=m.pH_modif == -log10((10 ** -m.pKw) / (10 ** -m.x1)))
    m.exCaOH_constr = Constraint(expr=m.exCaOH == ((10 ** (-m.x1)) / 2) * 74000)
#-------------------------------------------------------------------------------------------
"""

# if statement for type of softening
if m.Mg_CaCO3 <= 40 and m.Alk > m.Ca_CaCO3:
    print('1')
    m.CaOH_constr = Constraint(expr=m.CaOH == pyunits.convert(m.Q * (m.CO2_CaCO3 + m.CHCa) * 0.56, to_units=pyunits.kg/pyunits.d))
    m.CO2_req1_constr = Constraint(expr=m.CO2_req1 == m.Q * (m.Alk - m.Ca_CaCO3 + m.Ca_eff) * 0.44E-3)
    m.Na2CO3.fix(0)
    m.CO2_req2.fix(0)
    m.exCaOH.fix(0)
elif m.Mg_CaCO3 > 40 and m.TH <= m.Alk:
    print('2')
    m.exCaOH_constr = Constraint(expr=m.exCaOH == (m.CO2_CaCO3+m.Alk+m.Mg_CaCO3)*0.15)  
    m.CaOH_constr = Constraint(expr=m.CaOH == m.Q * (m.CO2_CaCO3 + m.Alk + m.Mg_CaCO3 + m.exCaOH) * 0.56E-3)
    m.CO2_req1_constr = Constraint(expr=m.CO2_req1 == m.Q * (m.Alk - m.TH - m.exCaOH + m.Ca_eff + 2*m.exCaOH + m.Mg_eff) * 0.44E-3)
    m.Na2CO3.fix(0)
    m.CO2_req2.fix(0)
    #m.exCaOH.fix()
elif m.Mg_CaCO3 <= 40 and m.TH > m.Alk:
    print('3')
    m.CaOH_constr = Constraint(expr=m.CaOH == pyunits.convert(m.Q * (m.CO2_CaCO3 + m.CHCa) * 0.56, to_units=pyunits.kg/pyunits.d))
    m.Na2CO3_constr = Constraint(expr=m.Na2CO3 == pyunits.convert(m.Q * (m.NCHCa + m.NCHMg) * 1.06, to_units=pyunits.kg/pyunits.d))
    m.CO2_req1_constr = Constraint(expr=m.CO2_req1 == m.Q * (m.Alk + (m.NCHCa+m.NCHMg) - m.Ca_CaCO3 + m.Ca_eff) * 0.44E-3)
    m.CO2_req2.fix(0)
    m.exCaOH.fix(0)
elif m.Mg_CaCO3 > 40 and m.TH > m.Alk:
    print('4')
    m.exCaOH_constr = Constraint(expr=m.exCaOH == (m.CO2_CaCO3+m.Alk+m.Mg_CaCO3)*0.15)  
    m.CaOH_constr = Constraint(expr=m.CaOH == pyunits.convert(m.Q * (m.CO2_CaCO3 + m.CHCa + 2*m.CHMg + m.NCHMg + m.exCaOH) * 0.56, to_units=pyunits.kg/pyunits.d))
    m.Na2CO3_constr = Constraint(expr=m.Na2CO3 == pyunits.convert(m.Q * (m.NCHCa + m.NCHMg) * 1.06, to_units=pyunits.kg/pyunits.d))
    m.CO2_req1_constr = Constraint(expr=m.CO2_req1 == m.Q * (m.exCaOH + m.Mg_eff) * 0.44E-3)
    m.CO2_req2_constr = Constraint(expr=m.CO2_req2 == m.Q * (m.Alk + (m.NCHCa+m.NCHMg) - m.TH + (m.Ca_eff+m.Mg_eff)) * 0.44E-3)
    #m.exCaOH.fix()


if silica_removal:
    if m.Si*2.35 > m.Mg and m.Si > 50:
        m.MgCl2_constr = Constraint(expr=m.MgCl2 == m.Q * (m.Si * 9.2) * 1e-3)
    else:
        m.MgCl2.fix(0)

m.Sludge_constr = Constraint(expr=m.Sludge == (m.Q * (2 * m.CHCa + 2.6 * m.CHMg + m.NCHCa + 1.6 * m.NCHMg + m.TSS + m.MgCl2 + m.exCaOH) ) * 1E-3)   

m.V1_constr = Constraint(expr=m.V1 == m.Q * (m.RT_R / 1440) * m.Num_R)
m.E1_constr = Constraint(expr=m.E1 == (m.G_R ** 2) * m.mu * m.V1)
m.V2_constr = Constraint(expr=m.V2 == m.Q * (m.RT_F / 1440) * m.Num_F)
m.E2_constr = Constraint(expr=m.E2 == (m.G_F ** 2) * m.mu * m.V2)
m.V3_constr = Constraint(expr=m.V3 ==  m.Q * (m.RT_S / 1440))
m.V4_constr = Constraint(expr=m.V4 == m.Q * (m.RT_Rec / 1440))
m.rapid_mix_power_constr = Constraint(expr=m.rapid_mix_power == m.E1 * 0.001 * 24)
m.floc_power_constr = Constraint(expr=m.floc_power == m.E2 * 0.001 * 24)

# Cost calculations

# Capital
if m.G_R <= 300:
    m.C1_constr = Constraint(expr=m.C1 == (0.0002 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) ** 2) + 22.776 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) + 28584) #35.315 to convert m3 to ft3
elif m.G_R > 300 and m.G_R <= 600:
    m.C1_constr = Constraint(expr=m.C1 == (0.0002 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) ** 2) + 29.209 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) + 30388) #35.315 to convert m3 to ft3
elif m.G_R > 600 and m.G_R <= 1000:
    m.C1_constr = Constraint(expr=m.C1 == (0.0002 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) ** 2) + 55.443 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) + 29756) #35.315 to convert m3 to ft3
else:
    print('ERROR - OUT OF BOUNDS (300 - 1000 s-1)')

if m.G_F <= 20:
    m.C2_constr = Constraint(expr=m.C2 == (566045 * (m.V2 / 3785) + 224745)) # Book - G=80 ---> ENR CCI = 8889 for Los Angeles, California (April 2007)
elif m.G_F > 20 and m.G_F <= 50:
    m.C2_constr = Constraint(expr=m.C2 == (673894 * (m.V2 / 3785) + 217222)) # Book - G=80 ---> ENR CCI = 8889 for Los Angeles, California (April 2007)
elif m.G_F > 50 and m.G_F <= 80:
    m.C2_constr = Constraint(expr=m.C2 == (952902 * (m.V2 / 3785) + 177335)) # Book - G=80 ---> ENR CCI = 8889 for Los Angeles, California (April 2007) 
else:
    print('ERROR - OUT OF BOUNDS (20 - 80 s-1)')

m.C3_constr = Constraint(expr=m.C3 == (- 0.0005 * ((pyunits.convert(m.V3,to_units=pyunits.ft**3))/(pyunits.convert(m.D_S,to_units=pyunits.ft))) ** 2) + 86.89 * ((m.V3 * 35.315)/(pyunits.convert(m.D_S,to_units=pyunits.ft))) + 182801)
m.C4_constr = Constraint(expr=m.C4 == (4e-9 * ((pyunits.convert(m.V4,to_units=pyunits.ft**3))**3)) - (0.0002 * (pyunits.convert(m.V4,to_units=pyunits.ft**3)) ** 2) + (10.027 * (pyunits.convert(m.V4,to_units=pyunits.ft**3))) + 19287)
m.C5_constr = Constraint(expr=m.C5 == (9e-8 * (pyunits.convert(m.CO2_req1+m.CO2_req2,to_units = pyunits.lb / pyunits.day))) - (0.001 * (pyunits.convert(m.CO2_req1+m.CO2_req2,to_units = pyunits.lb / pyunits.day)) ** 2) + (42.578 * (pyunits.convert(m.CO2_req1+m.CO2_req2,to_units = pyunits.lb / pyunits.day))) + 130812)
m.C6_constr = Constraint(expr=m.C6 == 20.065 * (pyunits.convert(m.CaOH,to_units = pyunits.lb / pyunits.hour)) + 193268)
m.C7_constr = Constraint(expr=m.C7 == 69195 * ((m.Q / 3785)**0.5523))


# Operational
if m.G_R <= 300:
    m.OC1_constr = Constraint(expr=m.OC1 == (-3e-8 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) ** 3) + (0.0008 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) ** 2) + (2.8375 * (pyunits.convert(m.V1,to_units=pyunits.ft**3))) + 22588)
elif m.G_R > 300 and m.G_R <= 600:
    m.OC1_constr = Constraint(expr=m.OC1 == (-3e-8 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) ** 3) + (0.0008 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) ** 2) + (7.8308 * (pyunits.convert(m.V1,to_units=pyunits.ft**3))) + 22588)
elif m.G_R > 600 and m.G_R <= 1000:
    m.OC1_constr = Constraint(expr=m.OC1 == 36.096 * (pyunits.convert(m.V1,to_units=pyunits.ft**3)) + 18928)
else:
    print('ERROR - OUT OF BOUNDS (300 - 1000 s-1)')

if m.G_F <= 20:
    m.OC2_constr = Constraint(expr=m.OC2 == (3e-13 * (pyunits.convert(m.V2,to_units=pyunits.ft**3)) ** 3) + (-5e-7  * (pyunits.convert(m.V2,to_units=pyunits.ft**3)) ** 2) + (0.2757 * (pyunits.convert(m.V2,to_units=pyunits.ft**3))) + 6594)
elif m.G_F > 20 and m.G_F <= 50:
    m.OC2_constr = Constraint(expr=m.OC2 == (3e-13 * (pyunits.convert(m.V2,to_units=pyunits.ft**3)) ** 3) + (-4e-7  * (pyunits.convert(m.V2,to_units=pyunits.ft**3)) ** 2) + (0.318 * (pyunits.convert(m.V2,to_units=pyunits.ft**3))) + 6040)
elif m.G_F > 50 and m.G_F <= 80:
    m.OC2_constr = Constraint(expr=m.OC2 == (-3e-7 * (pyunits.convert(m.V2,to_units=pyunits.ft**3)) ** 2) + (0.5692 * (pyunits.convert(m.V2,to_units=pyunits.ft**3))) + 6748)
else:
    print('ERROR - OUT OF BOUNDS (20 - 80 s-1)')

m.OC3_constr = Constraint(expr=m.OC3 == (7e-10 * ((pyunits.convert(m.V3,to_units=pyunits.ft**3))/(pyunits.convert(m.D_S,to_units=pyunits.ft))) ** 3) - (0.00005 * ((pyunits.convert(m.V3,to_units=pyunits.ft**3))/(pyunits.convert(m.D_S,to_units=pyunits.ft))) ** 2) + (1.5908 * ((pyunits.convert(m.V3,to_units=pyunits.ft**3))/(pyunits.convert(m.D_S,to_units=pyunits.ft)))) + 6872)
m.OC4_constr = Constraint(expr=m.OC4 == (1e-8 * ((pyunits.convert(m.CO2_req1+m.CO2_req2,to_units = pyunits.lb / pyunits.day))**3)) - (0.0004 * (pyunits.convert(m.CO2_req1+m.CO2_req2,to_units = pyunits.lb / pyunits.day)) ** 2) + (6.19 * (pyunits.convert(m.CO2_req1+m.CO2_req2,to_units = pyunits.lb / pyunits.day))) + 10265)
m.OC5_constr = Constraint(expr=m.OC5 == 4616.7 * ((pyunits.convert(m.CaOH,to_units = pyunits.lb / pyunits.day))**0.4589))
m.OC6_constr = Constraint(expr=m.OC6 == (pyunits.convert(m.Na2CO3,to_units=pyunits.kg / pyunits.year) * 0.65)) # adapted a cost of $0.65 per kg of Soda Ash
m.OC7_constr = Constraint(expr=m.OC7 == (pyunits.convert(m.MgCl2,to_units=pyunits.kg / pyunits.year) * 1.5)) # adapted a cost of $1.5 per kg of MgCl2
m.OC8_constr = Constraint(expr=m.OC8 ==(pyunits.convert(m.Sludge,to_units=pyunits.tons / pyunits.year) * 35)) # adapted a cost of $35 per ton of lime sludge and others
m.OC9_constr = Constraint(expr=m.OC9 == 88589 * ((m.Q / 3785)**0.4589))

#CAPEX and OPEX
m.CAPEX_constr = Constraint(expr=m.CAPEX == m.C1+m.C2+m.C3+m.C4+m.C5+m.C6+m.C7)
m.OPEX_constr = Constraint(expr=m.OPEX == m.OC1+m.OC2+m.OC3+m.OC4+m.OC5+m.OC6+m.OC7+m.OC8+m.OC9)

# NOTES:
# COSTS NEEDS TO BE SCALED FROM 2009$ --> CURRENT $

#FIXED VARIABLES
m.RT_F.fix()
m.RT_S.fix()
m.RT_Rec.fix()
m.RT_R.fix()
m.D_S.fix()
#m.G_R.fix()
#m.G_F.fix()
m.Num_R.fix()
m.Num_F.fix()
m.pH.fix()
m.Ca_eff.fix()
m.Mg_eff.fix()

#m.display()

dof = degrees_of_freedom(m)
print(f'\nDegrees of Freedom: {dof}\n')

solver = SolverFactory('ipopt')
# solver.options['tol'] = 1e-6
results = solver.solve(m)
print(f'Model solve = {results.solver.termination_condition}\n')

#print(m.exCaOH())

1

Degrees of Freedom: 0

Model solve = optimal



In [8]:
# Results cell

print('TH:',m.TH.value)
#print('CH:',m.CH.value)
#print('NCH:',m.NCH.value)

print('Carbonic acid concentration:',m.CO2_CaCO3.value, 'mg/L as CaCO3')
print('Carbonate Hardness due to Calcium:,',m.CHCa.value,'mg/L as CaCO3')
print('Carbonate Hardness due to Magnesium:,',m.CHMg.value,'mg/L as CaCO3')
print('Non-carbonate Hardness due to Calcium: ',m.NCHCa.value,'mg/L as CaCO3')
print('Non-carbonate Hardness due to Magnesium: ',m.NCHMg.value,'mg/L as CaCO3')
print('excess hydrated lime', m.exCaOH.value,'mg/L')

print()

print('Lime addition: ',m.CaOH.value,'kg/d')
print('Soda Ash addition: ',m.Na2CO3.value,'kg/d')
print('CO2 addition for pH adjustment after softenining (1): ',m.CO2_req1.value,'kg/d')
print('CO2 addition for pH adjustment after softenining (2): ',m.CO2_req2.value,'kg/d')
print('MgCl2 for silica removal: ',m.MgCl2.value,'kg/d')
print('Sludge produced',m.Sludge.value,'kg/d')

print()

print('Total volume of rapid mix tank: ',m.V1.value,'m3')
print('Power consumption in rapid mixing: ',m.rapid_mix_power.value,'kWh/d')
print('Total volume of flocculation tank: ',m.V2.value,'m3')
print('Power consumption in flocculation mixing: ',m.floc_power.value,'kWh/d')
print('Total volume of sedimentation basin: ',m.V3.value,'m3')
print('Total volume of recarbonation basin: ',m.V4.value,'m3')

print()

print('CC of rapid mix: $',m.C1.value)
print('CC of flocculation tank: $',m.C2.value)
print('CC of sedimentation basin - circular basin: $',m.C3.value)
print('CC of recarbonation basin: $',m.C4.value)
print('CC of recarbonation - liquid CO2 as CO2 source: $',m.C5.value)
print('CC of lime feed system: $',m.C6.value)
print('CC of administrative, laboratory, and maintenance buildings: $',m.C7.value)

print()

print('OC of rapid mix: $',m.OC1.value,'per year')
print('OC of flocculation tank: $',m.OC2.value,'per year')
print('OC of sedimentation basin - circular: $',m.OC3.value,'per year')
print('OC of recarbonation - liquid CO2 as CO2 source: $',m.OC4.value,'per year')
print('OC of lime feed system: $',m.OC5.value,'per year')
print('OC of soda ash chemical feed: $',m.OC6.value,'per year')
print('OC of magnesium chemical feed: $',m.OC7.value,'per year')
print('OC of lime sludge management: $',m.OC8.value,'per year')
print('OC of administrative, laboratory, and maintenance buildings: $',m.OC9.value,'per year')

print()

print('Total CAPEX: $', m.CAPEX.value)
print('Total OPEX: $', m.OPEX.value,'per year')

TH: 212.632
Carbonic acid concentration: 116.06400000000001 mg/L as CaCO3
Carbonate Hardness due to Calcium:, 187.5 mg/L as CaCO3
Carbonate Hardness due to Magnesium:, 8.5 mg/L as CaCO3
Non-carbonate Hardness due to Calcium:  0 mg/L as CaCO3
Non-carbonate Hardness due to Magnesium:  16.632000000000005 mg/L as CaCO3
excess hydrated lime 0 mg/L

Lime addition:  8499.791999999998 kg/d
Soda Ash addition:  0 kg/d
CO2 addition for pH adjustment after softenining (1):  847.0 kg/d
CO2 addition for pH adjustment after softenining (2):  0 kg/d
MgCl2 for silica removal:  0 kg/d
Sludge produced 31185.559999999998 kg/d

Total volume of rapid mix tank:  13.88888888888889 m3
Power consumption in rapid mixing:  39.1797 kWh/d
Total volume of flocculation tank:  1736.111111111111 m3
Power consumption in flocculation mixing:  136.04062499999998 kWh/d
Total volume of sedimentation basin:  4513.888888888889 m3
Total volume of recarbonation basin:  694.4444444444445 m3

CC of rapid mix: $ 39803.320656448865

In [6]:
# Variable value testing cell
print(m.K2())
print(m.K1())
print(m.Alpha_1())
print(m.CT_1())
print(m.CO2_CaCO3())

1e-10
1e-07
0.75
0.0025
116.06400000000001


In [ ]:
# print_infeasible_constraints(m)
# print_infeasible_bounds(m)
# print_variables_close_to_bounds(m)
# print_constraints_close_to_bounds(m)
# print_close_to_bounds(m)
# print(infeas.log_infeasible_bounds(m))
# print(infeas.log_infeasible_constraints(m))

Lime sludge disposal approximate costs per ton

https://www.whig.com/archive/article/quincy-has-1-6-million-backlog-of-water-sewer-sludge/article_63bcdbc5-ed52-55a6-94e4-7c7edeb8bc96.html

https://www.cityofames.org/home/showpublisheddocument/52089/637009599571670000

https://www.vicksburgpost.com/2002/10/21/lime-sludge-at-water-plant-a-building-problem-for-city/

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6069973/

excess lime can be simplified 

https://usa1lib.vip/book/593673/55e0f6

https://www.studocu.com/my/document/universiti-tun-hussein-onn-malaysia/civil-engineering/21-water-softening-calculating-calcium-h/1186173

https://www.egr.msu.edu/~hashsham/courses/ene806/docs/Water%20Softening%201.pdf

https://commons.und.edu/cgi/viewcontent.cgi?article=2309&context=theses
pg37
https://unitslab.com/node/2

##### Cost calculations

Ref 1: https://rc.library.uta.edu/uta-ir/handle/10106/4924

Ref 2: McGivney, W. T. & Kawamura, S. (2008) Cost Estimating Manual for Water Treatment Facilities. John Wiley & Sons, Inc., Hoboken, NJ, USA.